# Usar el modelo pre-entrenado VGG19 para clsificar el dataset CIFAR10

In [ ]:
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 12.3 MB/s eta 0:00:00


In [ ]:

import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn.functional as F
import torchvision.datasets as datasets
import torch.nn as nn
import numpy as np
from torchmetrics import Accuracy
from tqdm import tqdm
from torch.utils.data import DataLoader

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# GPU
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print('device:', device)

device: cuda:0


In [ ]:
# Guardar los nombres de las clases de CIFAR10
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Los de VGG serian
# weights = VGG19_Weights.DEFAULT
# clases = weights.meta["categories"]

### Preprocesar y preparar los datos

In [ ]:
# Definir transform para transformar las imgs a Tensores con tamanho  normalizados en rango que VGG acepta como entrada, 224x224
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomCrop(32, padding=4),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
## Cargar y Aplicar transformaciones a las imagenes, desde los datasets que ofrece pytorch
##  El conjunto de datos CIFAR10 consta de imágenes en color de 32x32 píxeles, con tres canales (rojo, verde y azul)
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Dividir el conjunto de entrenamiento en entrenamiento y evaluación
train_size = int(0.8 * len(train_dataset))
eval_size = len(train_dataset) - train_size
train_dataset, eval_dataset = torch.utils.data.random_split(train_dataset, [train_size, eval_size])

# Crear un dataloader para cargar las imágenes en lotes
batch_size = 32
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=2, shuffle=True)
eval_loader = torch.utils.data.DataLoader(eval_dataset, batch_size=batch_size, num_workers=2, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=2, shuffle=False)


100%|██████████| 170498071/170498071 [00:01<00:00, 106058435.18it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


## Cargar el modelo

In [ ]:
# Cargar el modelo VGG19 preentrenado con la última capa modificada
model = models.vgg19_bn(pretrained=True)
model.classifier[6] = nn.Linear(4096, 10)
model.to(device)

## Entrenar, evaluar y probar modelo. Guardarlo en la nube  

In [ ]:
def train(model,dataloader,num_epochs):
  # Definir la funcion de perdida y de optimizacion
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    model.train()

    for epoch in range(num_epochs):
        running_loss = 0.0
        num_iterations = 0
        progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))
        for i, (images, labels) in progress_bar:
            # Mover los datos a la GPU si es posible
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Acumular la pérdida promedio por epoch
            running_loss += loss.item()
            num_iterations += 1


            progress_bar.set_description(f"Epoch {epoch + 1}/{num_epochs}, Loss: {(running_loss / num_iterations):.4f}")
            running_loss = 0.0

        progress_bar.close()

    print('\n Training completed')

In [ ]:

def evaluate(model,dataloader):
    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        progress_bar = tqdm(dataloader, total=len(dataloader))
        for inputs, labels in progress_bar:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)
            _, predicted = torch.max(probabilities, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            progress_bar.set_description(f"Evaluación")
        progress_bar.close()

    # Calcular la precisión utilizando torchmetrics
    accuracy = Accuracy(task='MULTICLASS', num_classes=10)
    accuracy = accuracy(torch.tensor(all_predictions), torch.tensor(all_labels))

    print(f"Evaluate Accuracy: {(float(accuracy) * 100):.2f}%\n")


In [ ]:
def test(model,dataloader):
    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        progress_bar = tqdm(dataloader, total=len(dataloader))
        for inputs, labels in progress_bar:
            inputs = inputs.to(device)
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)
            _, predicted = torch.max(probabilities, 1)
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            progress_bar.set_description(f"Test")
        progress_bar.close()

    # Calcular la precisión utilizando torchmetrics
    accuracy = Accuracy(task='MULTICLASS', num_classes=10)
    accuracy = accuracy(torch.tensor(all_predictions), torch.tensor(all_labels))
    print(f"Test Accuracy: {(float(accuracy) * 100):.2f}%")

In [ ]:
# Entrenar el modelo
train(model,train_loader,4)

Epoch 4/4, Loss: 0.0000: 100%|██████████| 1250/1250 [12:56<00:00,  1.61it/s]

Entrenamiento completado


In [ ]:
# Evaluar el modelo en el conjunto de evaluación
evaluate(model,eval_loader)

Evaluación: 100%|██████████| 313/313 [01:05<00:00,  4.77it/s]

Evaluación Accuracy: 94.41%


In [ ]:
# Probar el modelo en el conjunto de prueba
test(model,test_loader)

Test: 100%|██████████| 313/313 [01:05<00:00,  4.77it/s]


Test Accuracy: 93.96%


In [ ]:
# Guardar modelo entrenado
torch.save(model, '/content/gdrive/My Drive/Colab Notebooks/modelCompleteVGG19.pth')